In [1]:
import boto3

dynamodb = boto3.resource('dynamodb',region_name='us-east-2') 

In [2]:
import pandas as pd
game_log = pd.read_csv('../Data/game_log.csv')

'''
before creating the dyanmoDB tables you need to know how you are going to use the table to design the appropriate 
keys and indexes to create efficient lookup structures. 

game_log: 
c: this is what we're tryna figure out
r: for read, we are going to give the table a date and retrieve games, the date doesn't have to be unique but the fucking games do
u: we won't be updating records but we will be adding records to the database daily with new games
d: We wont be deleting records because we need the entire game log duh?

in dyanmoDB we have:
Hash key: 
REQUIRED, for a single key table this is the only index you will have BUT this will limit you 
to only one read at a time with get item. Which is trash! and won't work if we are needing to index by date
to get back multiple games. additionally if we were to only use the single key of say game_id to actually get all the 
games we would have to perform a SCAN! we dont want to ever perform scans? are we a fax machine get that shit out of 
here. We want to create an effective index structure to pick the shit we need fast and get on with our lives. 

What can we do? 
COMPOSITE KEY! 
Here we have both 
HASH and SORT KEY:
Hash Key which is known as the partition key (for the game log case would be the GAME_DATE)

Sort key which is known as the range key ( for the game_log case would be the GAME_ID ) 

Having this combination key structure allows us to use the QUERY functionality of dynamoDB which is what 
we need to efficiently look for games based of game date but also identify games uniquely. 


Before we create this table and upload our data we have to ask a few questions:

For the game log table, will I EVER been querying/searching based of another ID? or column? ... 

The reason being, once you set your table structure with it's respective Keys and wotnot you cant go back! 

I REPEAT! once you set your table its done mate, you cant change that shit. 

So lets investigate the table
''' 

## Create GAME_LOG table

In [30]:
game_log = game_log.drop('_id', axis = 1)

In [63]:
game_log_ddb = dynamodb.create_table(
        TableName='game_log',
        KeySchema=[
            {
                'AttributeName': 'GAME_DATE',
                'KeyType': 'HASH'  # Partition key
            },
            {
                'AttributeName': 'GAME_ID',
                'KeyType': 'RANGE'
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'GAME_DATE',
                'AttributeType': 'S'
            },
            {
                'AttributeName': 'GAME_ID',
                'AttributeType': 'N'
            },

        ],
    BillingMode= 'PAY_PER_REQUEST',
    ) 

In [3]:
from decimal import Decimal
import json

game_log_dict = game_log.drop([7731, 7732]).to_dict(orient = 'records', )

#dropped because nan doesnt convert to decimal which is annoying!

In [65]:
game_log_json = [json.loads(json.dumps(item), parse_float=Decimal) for item in game_log_dict]

In [66]:
%%time
missed_indexes = []
with game_log_ddb.batch_writer() as batch:
    for i in range(len(game_log_json)):
        try:
            batch.put_item(Item = game_log_json[i])
        except:
            missed_indexes.append(i)

CPU times: user 4.72 s, sys: 109 ms, total: 4.83 s
Wall time: 22.4 s


# BEFORE WE CREATE MORE TABLES HOW THE FUCK DO WE QUERY THO LOL

In [67]:
from boto3.dynamodb.conditions import Key

pd.DataFrame(data = game_log_ddb.query(
    KeyConditionExpression=Key('GAME_DATE').eq('2022-01-01')
)['Items'])

## Create historical_pbp table

In [4]:
# historical_pbp = pd.read_csv('../Data/historical_pbp.csv')
historical_pbp_modelled = pd.read_csv('../Data/historical_pbp_modelled.csv')

# historical_pbp['PLAY_NUMBER'] = historical_pbp.groupby('GAME_ID').cumcount()

historical_pbp_modelled['PLAY_NUMBER'] = historical_pbp_modelled.groupby('GAME_ID').cumcount()

# historical_pbp = historical_pbp.drop('_id', axis = 1)
historical_pbp_modelled = historical_pbp_modelled.drop('_id', axis = 1)


In [75]:
#get rid of _id, 
historical_pbp = historical_pbp.astype(str).astype({'GAME_ID': int, 'PLAY_NUMBER':int})

In [5]:
historical_pbp_modelled_dict = historical_pbp_modelled[~historical_pbp_modelled['home_team_win'].isna()].to_dict('records')
historical_pbp_modelled_json = [json.loads(json.dumps(item), parse_float=Decimal) for item in historical_pbp_modelled_dict]

## How am I going to query the historical pbp


... just going to retrieve the play by play innit? like nothing really else so the composite key can be
GAME ID and PLAY_NUM


## I DONT NEED HISTORICAL_PBP

## HISTORICAL_PBP_MODLLED IS ENOUGH! 

In [80]:
# historical_pbp_ddb = dynamodb.create_table(
#         TableName='historical_pbp',
#         KeySchema=[
#             {
#                 'AttributeName': 'GAME_ID',
#                 'KeyType': 'HASH'  # Partition key
#             },
#             {
#                 'AttributeName': 'PLAY_NUMBER',
#                 'KeyType': 'RANGE'
#             }
#         ],
#         AttributeDefinitions=[
#             {
#                 'AttributeName': 'GAME_ID',
#                 'AttributeType': 'N'
#             },
#             {
#                 'AttributeName': 'PLAY_NUMBER',
#                 'AttributeType': 'N'
#             },

#         ],
#     BillingMode= 'PAY_PER_REQUEST',
#     ) 

In [79]:
historical_pbp_dict = historical_pbp.to_dict('records')

In [6]:
historical_pbp_modelled_ddb = dynamodb.create_table(
        TableName='historical_pbp_modelled',
        KeySchema=[
            {
                'AttributeName': 'GAME_ID',
                'KeyType': 'HASH'
            },
            {
                'AttributeName': 'PLAY_NUMBER',
                'KeyType': 'RANGE'
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'GAME_ID',
                'AttributeType': 'N'
            },
            {
                'AttributeName': 'PLAY_NUMBER',
                'AttributeType': 'N'
            },

        ],
    BillingMode= 'PAY_PER_REQUEST',
    ) 

In [7]:
len(historical_pbp_modelled_json)

3598846

In [9]:
%%time

with historical_pbp_modelled_ddb.batch_writer() as batch:
    for i in range(len(historical_pbp_modelled_json)):
        batch.put_item(Item = historical_pbp_modelled_json[i])

CPU times: user 19min 44s, sys: 51.1 s, total: 20min 35s
Wall time: 2h 25min 36s


In [ ]:
# https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/GSI.html